In [1]:
import pandas as pd
import math
import numpy as np
import math
import zipfile
import matplotlib
import geopandas
import lxml
import copy

from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [632]:
#Extract household income mean for districts in 2000
df_1 = pd.read_stata("sddbdist_2000.dta")
df_1 = df_1[["sddbid", "hhinc_median", "state", "name", "type", "stfips", "cofips", "leaid", "hhinc_mean"]]

In [4]:
#to get country population 1990 - 2014
pop_years_1 = ["pop1991","pop1992","pop1993","pop1994","pop1995","pop1996","pop1997","pop1998","pop1999"
            ,"pop2000","pop2001","pop2002","pop2003","pop2004","pop2005","pop2006","pop2007","pop2008"
            ,"pop2009"]

pop_years_2 = ["pop2010","pop2011", "pop2012"]

df_pop = pd.read_stata("county_population.dta")
#df_pop = df_pop[["fips", "state_fips", "county_fips", "country_name"]+pop_years]


#pop 2010 ...
df_pop1 = df_pop[df_pop["region"].notna()][df_pop["county_fips"]!=0].reset_index()

#pop < 2010
df_pop2 = df_pop[df_pop["pop2009"].notna()][df_pop["county_fips"]!=0].reset_index()


#df_pop1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [5]:
#finance 
years = [1990, 1992, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005
        , 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]

#list of sas files 
sas_files = [
    "finance/1990.sas7bdat","finance/1992.sas7bdat","finance/1995.sas7bdat","finance/1996.sas7bdat"
    ,"finance/1997.sas7bdat","finance/1998.sas7bdat","finance/1999.sas7bdat","finance/2000.sas7bdat"
    ,"finance/2001.sas7bdat","finance/2002.sas7bdat","finance/2003.sas7bdat","finance/2004.sas7bdat"
    ,"finance/2005.sas7bdat","finance/2006.sas7bdat","finance/2007.sas7bdat","finance/2008.sas7bdat"
    ,"finance/2009.sas7bdat","finance/2010.sas7bdat","finance/2011.sas7bdat","finance/2012.sas7bdat"
    ,"finance/2013.sas7bdat"
]

In [6]:
#To converted bytes into string columms
def convert(df):
    df = df
    str_df = df.select_dtypes([np.object])
    str_df = str_df.stack().str.decode('utf-8').unstack()
    for col in str_df:
        df[col] = str_df[col]
        
    return df

In [9]:
#To clean school revenue and expenditure data without relying on LEAID
def stat(file_path, stats):
    df = pd.read_sas(file_path)
    df = convert(df)
    
    #some sas files have CONUM column as district FIPs numbers, others have FIPSCO as FIPS numbers
    if "CONUM" in df.columns:
        
        #find PPR and PPE for each district. District without FIPS are ignored by default
        df = df.groupby(["CONUM", "FIPST"]).agg(stats)
        df["PPR"] = df["TOTALREV"] / df["V33"]
        df["PPE"] = df["TOTALEXP"] / df["V33"]
        
        df = df.dropna().sort_values(by="FIPST").reset_index()
        
        #rename CONUM to FIPSCO for clarity 
        df = df.rename(columns={"CONUM": "FIPSCO"})
    
    else:
        try:
            df = df.groupby(["FIPSCO", "FIPST"]).agg(stats)
            df["PPR"] = df["TOTALREV"] / df["V33"]
            df["PPE"] = df["TOTALEXP"] / df["V33"]
        
            df = df.dropna().sort_values(by="FIPST").reset_index()
            df["FIPSCO"] = df["FIPST"] + df["FIPSCO"]

        except:
            return "FIPSCO not in columns"
    
    df["YEAR"] = int(file_path[8:12])
    
    return df

#To clean school revenue and expenditure data, relying on LEAID
def stat_wo(file_path, stats):
    df = pd.read_sas(file_path)
    df = convert(df)
    

    df = df.groupby(["LEAID", "FIPST"]).agg(stats)
    df["PPR"] = df["TOTALREV"] / df["V33"]
    df["PPE"] = df["TOTALEXP"] / df["V33"]
        
    df = df.dropna().sort_values(by="FIPST").reset_index()

    
    df["YEAR"] = int(file_path[8:12])
    
    return df

In [10]:
#dont touch this
stats = {"V33": "sum", "TOTALREV": "sum", "TFEDREV": "sum", "TSTREV": "sum", "TLOCREV": "sum", "TOTALEXP": "sum"}

df1 = stat("finance/1990.sas7bdat", stats)
df2 = stat("finance/1992.sas7bdat", stats)
df3 = stat("finance/1995.sas7bdat", stats)
df4 = stat("finance/1996.sas7bdat", stats)
df5 = stat("finance/1997.sas7bdat", stats)
df6 = stat("finance/1998.sas7bdat", stats)
df7 = stat("finance/1999.sas7bdat", stats)
df8 = stat("finance/2000.sas7bdat", stats)
df9 = stat("finance/2001.sas7bdat", stats)
df10 = stat("finance/2002.sas7bdat", stats)
df11 = stat("finance/2003.sas7bdat", stats)
df12 = stat("finance/2004.sas7bdat", stats)
df13 = stat("finance/2005.sas7bdat", stats)
df14 = stat("finance/2006.sas7bdat", stats)
df15 = stat("finance/2007.sas7bdat", stats)
df16 = stat("finance/2008.sas7bdat", stats)
df17 = stat("finance/2009.sas7bdat", stats)
df18 = stat("finance/2010.sas7bdat", stats)
df19 = stat("finance/2011.sas7bdat", stats)
df20 = stat("finance/2012.sas7bdat", stats)
df21 = stat("finance/2013.sas7bdat", stats)

df_list = [
    df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12,
    df13, df14, df15, df16, df17, df18, df19, df20, df21
]
    

In [64]:
d1 = stat_wo("finance/1990.sas7bdat", stats)
d2 = stat_wo("finance/1992.sas7bdat", stats)
d3 = stat_wo("finance/1995.sas7bdat", stats)
d4 = stat_wo("finance/1996.sas7bdat", stats)
d5 = stat_wo("finance/1997.sas7bdat", stats)
d6 = stat_wo("finance/1998.sas7bdat", stats)
d7 = stat_wo("finance/1999.sas7bdat", stats)
d8 = stat_wo("finance/2000.sas7bdat", stats)
d9 = stat_wo("finance/2001.sas7bdat", stats)
d10 = stat_wo("finance/2002.sas7bdat", stats)
d11 = stat_wo("finance/2003.sas7bdat", stats)
d12 = stat_wo("finance/2004.sas7bdat", stats)
d13 = stat_wo("finance/2005.sas7bdat", stats)
d14 = stat_wo("finance/2006.sas7bdat", stats)
d15 = stat_wo("finance/2007.sas7bdat", stats)
d16 = stat_wo("finance/2008.sas7bdat", stats)
d17 = stat_wo("finance/2009.sas7bdat", stats)
d18 = stat_wo("finance/2010.sas7bdat", stats)
d19 = stat_wo("finance/2011.sas7bdat", stats)
d20 = stat_wo("finance/2012.sas7bdat", stats)
d21 = stat_wo("finance/2013.sas7bdat", stats)

d_list = [
    d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12,
    d13, d14, d15, d16, d17, d18, d19, d20, d21
]

In [449]:
#main dataframe for financial numbers based on FIPST
df_main = pd.concat(df_list)
df_main["FIPST"] = df_main["FIPST"].astype(int)

In [450]:
#state aggregate over the years
stats_2 = stats
stats_2["PPR"] = "sum"
stats_2["PPE"] = "sum"
df_main = df_main.groupby(["FIPST", "YEAR"]).agg(stats_2).reset_index()
df_main["PPR"] = df_main["TOTALREV"]/df_main["V33"]
df_main["PPE"] = df_main["TOTALEXP"]/df_main["V33"]

#drop DC
df_main = df_main[df_main["FIPST"]!=11]

In [457]:
#event list
df_events = pd.read_excel("eventlist.xlsx")
df_events = df_events[:-4]

states_with_events = df_events.groupby("fips").sum().index.astype(int).values
states_with_events = list(states_with_events)

#read cpi
cpi = pd.read_excel("CPI.xlsx")[["YEAR", "REAL"]]

#leaid
df_id = pd.read_csv("LEAID.txt", sep=",", thousands=",", encoding="latin-1")

#get state fipst
df_state = pd.read_stata("state_fips.dta")
df_state = df_state.rename(columns={"fips": "FIPST"})

In [452]:
#extracting values for no-reform and reform states
df_reform = df_main[df_main["FIPST"].isin(states_with_events)].reset_index().drop(axis=1, columns="index")
df_wo_reform = df_main[df_main["FIPST"].isin(states_with_events) == False].reset_index().drop(axis=1, columns="index")

In [454]:
#adjust for inflation
def adjust(df, cpi, var, code):
    assert code in ["FIPST", "LEAID"]
    
    df_adjust1 = pd.merge(df, cpi, left_on="YEAR", right_on="YEAR", how="left")
    df_adjust2 = df_adjust1[var].mul(df_adjust1.REAL, axis=0)
    if code=="FIPST":
        df_adjust = df_adjust1[[code, "YEAR", "V33"]].merge(df_adjust2, how="outer", left_index=True, right_index=True)
        return df_adjust
    df_adjust = df_adjust1[[code, "FIPST", "YEAR", "V33"]].merge(df_adjust2, how="outer", left_index=True, right_index=True)
    return df_adjust

In [476]:
var = ["TOTALREV", "TFEDREV", "TSTREV", "TLOCREV", "TOTALEXP", "PPR", "PPE"]
df_reform = adjust(df_reform, cpi, var, code="FIPST")
df_wo_reform = adjust(df_wo_reform, cpi, var, code="FIPST")
df_main = adjust(df_main, cpi, var, code="FIPST")

In [456]:
#To CSV
df_reform.to_csv("reform.csv")
df_wo_reform.to_csv("notreform.csv")

In [21]:
#fig, ax = plt.subplots(figsize=(7,4))
#ax = df_reform.plot(kind="scatter", y="PPR", x="YEAR", ax=ax)

#r = LinearRegression()
#r.fit(df_reform[["PPR"]].values, df["YEAR"].values.reshape(-1, 1))
#slope = float(r.coef_)
#intercept = float(r.intercept_)

#y0 = ax.get_xlim()[0]*slope + intercept
#y1 = ax.get_xlim()[1]*slope + intercept

#plt.plot(list(ax.get_xlim()), [y0, y1], color="red")

In [631]:
#Household income 1990
df_inc = pd.read_stata("getsddb1990_2015.dta")

df_inc = df_inc[["disttype", "state", "dist", "meanincA"]]
list_dist = list(df_inc["dist"])
#df_inc

In [462]:
#extract math or reading score for 4 grade pupils at the state-level for 50 states
def clean_score(filename, scorename):
    assert (scorename == "READING") or (scorename == "MATH")
    assert filename[-3:] == "csv"

    
    #keep only state names and scores
    df = pd.read_csv("scores/"+filename)[["Jurisdiction", "MN"]]

    #drop DoDEA, Puerto Rico, and District of Columbia, and National Public for data consistency
    df = df[df["Jurisdiction"] != "DoDEA"][df["Jurisdiction"] != "Puerto Rico"][df["Jurisdiction"] != "District of Columbia"][df["Jurisdiction"] != "National public"]
    
    df.index = range(len(df["MN"]))
    df = df.sort_values(by="Jurisdiction").reset_index(drop="True") #sort by state names and then reset indices
    df.columns = ["STATES", scorename] #rename columns to jurisdiction and Math or Reading scores
    
    #create year column
    if (int(filename[4:6]) >= 0) and (int(filename[4:6]) <= 13):
        df["YEAR"] = int("20"+filename[4:6])
    else:
        df["YEAR"] = int("19"+filename[4:6])
        
    assert df.shape[0] == 50
    
    return df

In [463]:
reading_files = ["read92.csv", "read94.csv", "read98.csv", "read02.csv", "read03.csv",
                 "read05.csv", "read07.csv", "read09.csv", "read11.csv", "read13.csv"]
math_files = ["math92.csv", "math96.csv", "math00.csv", "math03.csv",
              "math05.csv", "math07.csv", "math09.csv", "math11.csv", "math13.csv"] 

In [464]:
d = clean_score(reading_files[0], "READING").merge(df_state, right_on="name", left_on="STATES", how="left")
#combine reading and math scores

#create variables for reading score dataframes
b1 = clean_score(reading_files[0], "READING")
b2 = clean_score(reading_files[1], "READING")
b3 = clean_score(reading_files[2], "READING")
b4 = clean_score(reading_files[3], "READING")
b5 = clean_score(reading_files[4], "READING")
b6 = clean_score(reading_files[5], "READING")
b7 = clean_score(reading_files[6], "READING")
b8 = clean_score(reading_files[7], "READING")
b9 = clean_score(reading_files[8], "READING")
b10 = clean_score(reading_files[9], "READING")

#create variables for math score dataframes
c1 = clean_score(reading_files[0], "MATH")
c2 = clean_score(reading_files[1], "MATH")
c3 = clean_score(reading_files[2], "MATH")
c4 = clean_score(reading_files[3], "MATH")
c5 = clean_score(reading_files[4], "MATH")
c6 = clean_score(reading_files[5], "MATH")
c7 = clean_score(reading_files[6], "MATH")
c8 = clean_score(reading_files[7], "MATH")
c9 = clean_score(reading_files[8], "MATH")

b = [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10]
c = [c1, c2, c3, c4, c5, c6, c7, c8, c9]
#concatenate math and reading dataframes:
base_reading = pd.concat(b)
base_math = pd.concat(c)

df_reading = base_reading.merge(df_state, right_on="name", left_on="STATES", how="left")[["STATES", "READING", "YEAR", "FIPST"]]
df_math = base_math.merge(df_state, right_on="name", left_on="STATES", how="left")[["STATES", "MATH", "YEAR", "FIPST"]]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


In [465]:
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

#removing nan values for reading scores
df_reading["READING"] = df_reading[df_reading.applymap(isnumber)]["READING"]
df1 = df_reading.dropna().reset_index()
df1["READING"] = df1["READING"].astype(int)
df_reading_1 = df1.groupby(["STATES", "YEAR"]).mean().reset_index().drop(axis=1, columns="index")

df_reading_1["REFORM"] = np.where((df_reading_1.FIPST.isin(states_with_events)), 1, 0)


#removing nan values for math scores
df_math["MATH"] = df_math[df_math.applymap(isnumber)]["MATH"]
df2 = df_math.dropna().reset_index()
df2["MATH"] = df2["MATH"].astype(int)
df_math_1 = df2.groupby(["STATES", "YEAR"]).mean().reset_index().drop(axis=1, columns="index")
#df_math_1["REFORM"] = 1

df_math_1["REFORM"] = np.where((df_math_1.FIPST.isin(states_with_events)), 1, 0)

In [466]:
math_reform = df_math_1[df_math_1["FIPST"].isin(states_with_events)]
math_wo_reform = df_math_1[df_math_1["FIPST"].isin(states_with_events)==False]

reading_reform = df_reading_1[df_reading_1["FIPST"].isin(states_with_events)]
reading_wo_reform = df_reading_1[df_reading_1["FIPST"].isin(states_with_events)]

#math_reform

In [467]:
#Creat csv files for scores
reading_reform.to_csv("w_reading.csv")
reading_wo_reform.to_csv("wo_reading.csv")

math_reform.to_csv("w_math.csv")
math_wo_reform.to_csv("wo_math.csv")

df_math_1.to_csv("math.csv")
df_reading_1.to_csv("reading.csv")

In [577]:
##########################################################
##########################################################
##########################################################
##########################################################

In [634]:
#main dataframe based on LEAID
d_main = pd.concat(d_list)
d_main["FIPST"] = d_main["FIPST"].astype(int)
d_main = d_main[(d_main["TOTALREV"]>0) & (d_main["V33"]>0)]

#d_main = d_main[d_main["YEAR"]==1990]

d_main = d_main.merge(df_1, right_on="leaid", left_on="LEAID").drop(axis=1, columns=["state", "name", "type", "cofips", "stfips", "leaid", "sddbid"])
d_main = d_main.dropna().reset_index().drop(axis=1, columns="index")

var = ["TOTALREV", "TFEDREV", "TSTREV", "TLOCREV", "TOTALEXP", "PPR", "PPE", "hhinc_median", "hhinc_mean"]
d_main = adjust(d_main, cpi, var, code="LEAID")

In [615]:
d_main_year = d_main[d_main["YEAR"]==1990]
s1 = d_main_year.groupby(["FIPST"])["hhinc_mean"].quantile(0.05)
s2 = d_main_year.groupby(["FIPST"])["hhinc_mean"].quantile(0.95)

In [633]:
d_main1 = d_main_year.merge(s1, left_on="FIPST", right_on="FIPST").merge(s2, left_on="FIPST", right_on="FIPST")
d_main1 = d_main1.rename(columns={"hhinc_median": "HHINC_MEDIAN", "hhinc_mean_x": "HHINC_MEAN", "hhinc_mean_y": "Q1", "hhinc_mean": "Q5"})
d_main1["REFORM"] = np.where((d_main1.FIPST.isin(states_with_events)), 1, 0)

In [620]:
#To find counties below q1 and above q5
inc1 = d_main1[d_main1["HHINC_MEAN"]<d_main1["Q1"]]
inc2 = d_main1[d_main1["HHINC_MEAN"]>d_main1["Q5"]]

list_leaid1 = list(inc1["LEAID"])
list_leaid2 = list(inc2["LEAID"])

In [635]:
d_income1 = copy.deepcopy(d_main[d_main["LEAID"].isin(list_leaid1)])
d_income1["REFORM"] = np.where((d_income1.FIPST.isin(states_with_events)), 1, 0)
d_income2 = copy.deepcopy(d_main[d_main["LEAID"].isin(list_leaid2)])
d_income2["REFORM"] = np.where((d_income2.FIPST.isin(states_with_events)), 1, 0)

In [622]:
#with reforms 

wreform_q1 = d_income1[d_income1["REFORM"]==1]
wreform_q5 = d_income2[d_income2["REFORM"]==1]

mean1 = wreform_q1.groupby(["YEAR"]).mean().reset_index()
mean2 = wreform_q5.groupby(["YEAR"]).mean().reset_index()

#without reforms 

woreform_q1 = d_income1[d_income1["REFORM"]==0]
woreform_q5 = d_income2[d_income2["REFORM"]==0]

mean3 = woreform_q1.groupby(["YEAR"]).mean().reset_index()
mean4 = woreform_q5.groupby(["YEAR"]).mean().reset_index()

dif1 = (mean1-mean2)["PPR"]
dif2 = (mean3-mean4)["PPR"]


f1 = copy.deepcopy(mean1[["YEAR"]])
f1["DIFFERENCE"] = dif1
f1["REFORM"] = 1

f2 = copy.deepcopy(mean1[["YEAR"]])
f2["DIFFERENCE"] = dif2
f2["REFORM"] = 0

f3 = pd.concat([f1, f2])

In [623]:
f3.to_csv("difference.csv")